In [94]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [95]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GEMINI_API_KEY)


In [96]:
response = model.invoke("How do tiebreakers get hadnled in the FNCS?")

In [97]:
response

AIMessage(content="Here's how tiebreakers are handled in the FNCS, broken down by stage:\n\n**Qualifiers (Weeklies & Heats)**\n\n* **Placement is prioritized:**  If players have the same points, the player with the higher placement in the final match of the qualifier will advance.\n* **If placements are tied:** The player with the higher total eliminations across all matches in the qualifier will advance.\n\n**Finals**\n\n* **Placement is prioritized:**  If players have the same points, the player with the higher placement in the final match of the finals will win. \n* **If placements are tied:**  The player with the higher total eliminations across all matches in the finals will win.\n\n**Important Notes:**\n\n* **Double-Elimination:** In some FNCS formats, there are double-elimination brackets, meaning that players can lose one match and still have a chance to advance. Tiebreakers might be used within these brackets as well.\n* **Specific Rules:** The exact tiebreaker rules can vary 

In [104]:
os.chdir("../langchain_intro")

In [108]:
os.getcwd()

'c:\\Users\\Sosuke\\chat_bot\\langchain_intro'

In [109]:
# Clean the text file

import re

with open('fncs.txt', 'r', encoding='utf-8') as file:
    content = file.read()
content = content.replace("(", "").replace(")", "").replace("\n", " ")
pattern = r'\d{1}\.\d*\s'

result = re.split(pattern, content)
print(result)


['FNCS COMMUNITY CUPS - CHAPTER 5 SEASON 3 - OFFICIAL RULES 7.1', 'The Competitive Fortnite Team FNCS Community Cups - Chapter 5 Season 3 - Official Rules    These are the Official Rules “Rules” for the Chapter 5 Season 2 Fortnite FNCS Community Cups “FNCS Cup” or “Event”, which is hosted by or on behalf of Epic Games, Inc. “Epic”. These Rules are a legal agreement between you and Epic for your participation in the Event.    Table of Contents    ', 'Introduction and Acceptance  ', 'Event Structure  ', 'Player Eligibility; Epic Account Structure  ', 'Team Rules  ', 'Issues  ', 'Communication  ', 'Post-Match Process  ', 'Code of Conduct  ', 'Enforcement; Rules and Conduct Violations  1', 'Disclaimers  1', 'Publicity; Interview Consent  1', 'Governing Law  1', 'Waiver of Jury Trial  1', 'Privacy  1', 'Other Languages  Attachment A: Scoring System  Attachment B: Schedule  Attachment C: Event Prizes    ', 'Introduction and Acceptance    ', 'Introduction.  These Rules are designed to protect

In [111]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

template = """
Answer the question based on the context below using an example. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [112]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)

In [113]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore1 = DocArrayInMemorySearch.from_texts(
    result,
    embedding=embeddings,
)

In [114]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever1 = vectorstore1.as_retriever()
chain = (
    {"context": vectorstore1.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("How do tiebreakers get hadnled in the FNCS?")

'Tiebreakers in the FNCS are determined in the following order:\n\n1. **Total points scored**\n2. **Total Victory Royales in the session**\n3. **Average eliminations in the session**\n4. **Average placement per match in the session**\n5. **Total seconds survived across all matches**\n\n**Example:**\n\nIf two players have the same total points scored, the player with more Victory Royales would win the tiebreaker. If they have the same number of Victory Royales, the player with the higher average eliminations would win, and so on. \n'